In [10]:
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command
from langgraph.graph.message import MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage
from langchain.chat_models import init_chat_model
from typing import Literal


In [11]:
class AgentsState(MessagesState):
    current_agent: str
    transfered_by: str


llm = init_chat_model(model="openai:gpt-5-nano-2025-08-07")

In [ ]:
def make_agent(prompt, agent_name, tools):
    def agent_node(state: AgentsState):
        llm_with_tools = llm.bind_tools(tools)
        messages = [SystemMessage(content=prompt)] + state["messages"]
        response = llm_with_tools.invoke(messages)
        # response = llm_with_tools.invoke(
        #     f"""
        # {prompt}

        # Conversation History:
        # {state["messages"]}
        # """
        # )
        return {"messages": [response]}

    agent_builder = StateGraph(AgentsState)

    agent_builder.add_node("agent", agent_node)
    agent_builder.add_node("tools", ToolNode(tools=tools))

    agent_builder.add_edge(START, "agent")
    agent_builder.add_conditional_edges("agent", tools_condition)
    agent_builder.add_edge("tools", "agent")

    return agent_builder.compile()

In [13]:
@tool
def handoff_tool(
    transfer_to: Literal["korean_agent", "greek_agent", "english_agent"],
    transfered_by: str,
):
    """
    Handoff to another agent.
    Use this tool when the customer speaks a language that you don't understand.
    """
    return Command(
        update={
            "current_agent": transfer_to,
            "transfered_by": transfered_by,
        },
        goto=transfer_to,
        graph=Command.PARENT,  # 현재 그래프가 아닌, 한단계 위의 그래프로 통제권을 넘긴다
    )

In [14]:
graph_builder = StateGraph(AgentsState)

# 각 에이전트 노드 생성
agents = [
    {
        "name": "korean_agent",
        "prompt": "You are a Korean customer support agent. You only speak and understand Korean. If user speaks other languages, handoff to relevant agent.",
    },
    {
        "name": "greek_agent",
        "prompt": "You are a Greek customer support agent. You only speak and understand Greek. If user speaks other languages, handoff to relevant agent.",
    },
    {
        "name": "english_agent",
        "prompt": "You are an English customer support agent. You only speak and understand English. If user speaks other languages, handoff to relevant agent.",
    },
]

for agent_info in agents:
    graph_builder.add_node(
        agent_info["name"],
        make_agent(
            prompt=agent_info["prompt"],
            agent_name=agent_info["name"],
            tools=[handoff_tool],
        ),
    )

# 시작점 설정
graph_builder.add_edge(START, "korean_agent")

graph = graph_builder.compile()

TypeError: make_agent() got an unexpected keyword argument 'agent_name'

In [ ]:
for event in graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "안녕하세요. 영어로 대화해주세요.",
            },
        ]
    },
    stream_mode="updates",
):
    print(event)
